# Aplicando CNN para classificação de fissura em concreto armado e pavimento

Nestes notebooks será aplicado redes pré-treinadas para detecção de fissura, neste dataset de imagens, o foco é comparar a redes pré-treinadas e ver qual seria a rede com o melhor desempenho

In [1]:
#importando as blibiotecas

from tensorflow import keras
from keras.layers import Dense, Flatten
from tensorflow.keras.models import Model, Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import VGG16, ResNet50

import tensorflow as tf

In [2]:
# importando a blibioteca split-folders para dividir as imagens em treino e teste
!pip install split-folders

import splitfolders
splitfolders.ratio('/kaggle/input/concrete-and-pavement-crack-images', 
                   output="output", seed=1337, ratio=(.7, 0.3))

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Copying files: 30000 files [00:36, 825.86 files/s] 


In [3]:
BATCH = 26
IMG_WIDTH = 96
IMG_HEIGHT = 96

In [4]:
train_loader = tf.keras.preprocessing.image_dataset_from_directory(
"./output/train",
seed=123,
image_size=(IMG_HEIGHT, IMG_WIDTH),
batch_size=BATCH,label_mode= 'categorical'
)

test_loader = tf.keras.preprocessing.image_dataset_from_directory(
"./output/val",
seed=123,
image_size=(IMG_HEIGHT, IMG_WIDTH),
batch_size=BATCH, label_mode= 'categorical')

Found 21000 files belonging to 2 classes.
Found 9000 files belonging to 2 classes.


## Criando o modelo VGG16

In [5]:
model = VGG16(input_shape=(96, 96, 3), weights='imagenet', include_top=False,
             pooling='max',classifier_activation='softmax')
model.layers[0].trainable = False
model = Model(inputs=model.input, outputs=model.get_layer(index=-2).output)

In [6]:
#Colocando as duas últimas camadas
newTop = Sequential()
newTop.add(Flatten(input_shape=model.output_shape[1:]))
newTop.add(Dense(2, activation='softmax'))

/opt/conda/lib/python3.10/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [7]:
metricas = ['accuracy',
            keras.metrics.Precision(name='Pre'),keras.metrics.Recall(name='Rec')] 
#métricas que vão ser utilizadas para teste

In [8]:
model = Model(inputs=model.input, outputs=newTop(model.output))
model.summary()

optimizer = 'adam'
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=metricas)

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 96, 96, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 96, 96, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 96, 96, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 48, 48, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 48, 48, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 48, 48, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 24, 24, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 24, 24, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 24, 24, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 24, 24, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 12, 12, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 12, 12, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 12, 12, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 12, 12, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 6, 6, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 6, 6, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 6, 6, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 6, 6, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 3, 3, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 2)              │         9,218 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,723,906 (56.17 MB)

 Trainable params: 14,723,906 (56.17 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_loader.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
test_dataset = test_loader.cache().prefetch(buffer_size=AUTOTUNE)

In [12]:
callback = keras.callbacks.EarlyStopping(monitor='loss',
                                              patience=2)
history = model.fit(train_dataset,epochs=5,batch_size=100,callbacks=[callback])

Epoch 1/5
808/808 ━━━━━━━━━━━━━━━━━━━━ 37s 45ms/step - Pre: 0.9117 - Rec: 0.9117 - accuracy: 0.9117 - loss: 0.2535
Epoch 2/5
808/808 ━━━━━━━━━━━━━━━━━━━━ 36s 45ms/step - Pre: 0.9577 - Rec: 0.9577 - accuracy: 0.9577 - loss: 0.1222
Epoch 3/5
808/808 ━━━━━━━━━━━━━━━━━━━━ 36s 45ms/step - Pre: 0.9660 - Rec: 0.9660 - accuracy: 0.9660 - loss: 0.0991
Epoch 4/5
808/808 ━━━━━━━━━━━━━━━━━━━━ 37s 45ms/step - Pre: 0.9857 - Rec: 0.9857 - accuracy: 0.9857 - loss: 0.0466
Epoch 5/5
808/808 ━━━━━━━━━━━━━━━━━━━━ 36s 45ms/step - Pre: 0.9860 - Rec: 0.9860 - accuracy: 0.9860 - loss: 0.0453


In [13]:
model.evaluate(test_dataset)

347/347 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - Pre: 0.9798 - Rec: 0.9798 - accuracy: 0.9798 - loss: 0.0550


[0.05732189863920212,
 0.9794444441795349,
 0.9794444441795349,
 0.9794444441795349]